In [1]:
from atg import *

In [ ]:
PATH=""

In [10]:
os.chdir(PATH)
db_params=pickle.load(open(f"{PATH}/dicts/db_params.pickle", 'rb'))
cities_list=query4table('cities_boundary','public', db_params)['city']

In [5]:
for ind, city in enumerate(cities_list):
    print(ind)
    print(city)
    df=pd.read_csv(f"{PATH}/distances/{city}.dist.bz2")
    df['city']=city
    ### Generate geodesic distances
    df['geodesic_meters']=df.apply(lambda x: geodesic((x['lat_source'], x['long_source']),(x['lat_dest'], x['long_dest'])).km*1000, axis=1)
    #Transform in minutes assuming 5km/h converting factor
    df['geodesic_minutes']=df['geodesic_meters']*(60)/5000
    #keep only relevant columns
    df=df[['city', 'x_source', 'y_source', 'x_dest', 'y_dest', 'walk_minutes', 'geodesic_meters','geodesic_minutes']]
    #Round to integer
    for VAR in ['walk_minutes', 'geodesic_meters','geodesic_minutes']:
        df[VAR]=np.round(df[VAR], 1)
    df2psql(df, f"{city}", db_params, if_exists='replace', index=False, index_label=[ 'city', 'x_source','y_source', 'x_dest','y_dest'], schema='distances')